In [1]:
import requests
import pandas as pd
import io
from datetime import datetime
import os

In [2]:
#URLS datasets

url_museos = "https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/4207def0-2ff7-41d5-9095-d42ae8207a5d/download/museos_datosabiertos.csv"
url_cines = "https://datos.cultura.gob.ar/dataset/0560ef96-55ca-4026-b70a-d638e1541c05/resource/b8bf0459-16a5-430b-b8e1-4fb786572469/download/salas_cine.csv"
url_bibliotecas = "https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/01c6c048-dbeb-44e0-8efa-6944f73715d7/download/biblioteca_popular.csv"

In [3]:
#Obtencion fecha actual

today = datetime.now()
year_month = today.strftime('%Y-%B')
today, year_month

(datetime.datetime(2023, 4, 3, 10, 50, 35, 58962), '2023-April')

In [4]:
#Funcion para extraer dataset a travez de la url

def extract(url, tipo):
    folder = os.path.join(tipo, year_month)
    os.makedirs(folder, exist_ok=True)
    filename = f"{tipo}-{today.strftime('%d-%m-%Y')}.csv"
    filepath = os.path.join(folder, filename)
    r = requests.get(url)
    with open(filepath, 'wb') as f:
        f.write(r.content)
    return filepath

In [5]:
#EXTRACCION

path_museos = extract(url_museos, "museos")
path_bibliotecas = extract(url_bibliotecas, "bibliotecas")
folder = os.path.join("cines", year_month)
filename = f"cines-{today.strftime('%d-%m-%Y')}.csv"
path_cines = os.path.join(folder, filename)
path_museos, path_bibliotecas, path_cines

('museos\\2023-April\\museos-03-04-2023.csv',
 'bibliotecas\\2023-April\\bibliotecas-03-04-2023.csv',
 'cines\\2023-April\\cines-03-04-2023.csv')

In [6]:
#TRANSFORMACION

#Normalizacion de tablas

columns_norm_names=["cod_localidad", "id_provincia", "id_departamento","categoría",
              "provincia","localidad","nombre","domicilio","código postal","número de teléfono","mail","web"]

columns_museos_rename = {
    'Cod_Loc':'cod_localidad',
    'IdProvincia':'id_provincia',
    'IdDepartamento':'id_departamento',
    'categoria':'categoría',
    'direccion':'domicilio',
    'CP':'código postal',
    'telefono':'número de teléfono',
    'Mail':'mail',
    'Web':'web'
}
museos = pd.read_csv(path_museos)
museos.rename(columns=columns_museos_rename, inplace=True)
museosnorm =museos[columns_norm_names]

columns_cines_rename = {
    'Cod_Loc':'cod_localidad',
    'IdProvincia':'id_provincia',
    'IdDepartamento':'id_departamento',
    'Categoría':'categoría',
    'Dirección':'domicilio',
    'CP':'código postal',
    'Teléfono':'número de teléfono',
    'Mail':'mail',
    'Web':'web',
    'Provincia':'provincia',
    'Localidad':'localidad',
    'Nombre':'nombre',   
}

cines = pd.read_csv(path_cines)
cines.rename(columns=columns_cines_rename, inplace=True)
cinesnorm = cines[columns_norm_names]

columns_bibliotecas_rename = {
    'Cod_Loc':'cod_localidad',
    'IdProvincia':'id_provincia',
    'IdDepartamento':'id_departamento',
    'Categoría':'categoría',
    'Domicilio':'domicilio',
    'CP':'código postal',
    'Teléfono':'número de teléfono',
    'Mail':'mail',
    'Web':'web',
    'Provincia':'provincia',
    'Localidad':'localidad',
    'Nombre':'nombre',
}
bibliotecas = pd.read_csv(path_bibliotecas)
bibliotecas.rename(columns=columns_bibliotecas_rename, inplace=True)
bibliotecasnorm = bibliotecas[columns_norm_names]
cines = cines["cod_localidad", "id_provincia"]
cines.info()

KeyError: ('cod_localidad', 'id_provincia')

In [7]:
#Concatenacion de los df

df = pd.concat([museosnorm, cinesnorm, bibliotecasnorm], axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3528 entries, 0 to 2016
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   cod_localidad       3528 non-null   int64 
 1   id_provincia        3528 non-null   int64 
 2   id_departamento     3528 non-null   int64 
 3   categoría           3528 non-null   object
 4   provincia           3528 non-null   object
 5   localidad           3528 non-null   object
 6   nombre              3528 non-null   object
 7   domicilio           3510 non-null   object
 8   código postal       3462 non-null   object
 9   número de teléfono  3334 non-null   object
 10  mail                2610 non-null   object
 11  web                 404 non-null    object
dtypes: int64(3), object(9)
memory usage: 358.3+ KB


In [8]:
#CANTIDAD DE REGISTROS TOTALES POR CATEGORIA
size_by_category = df.groupby("categoría", as_index=False).size()
size_by_category.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   categoría  3 non-null      object
 1   size       3 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 176.0+ bytes


In [90]:
#CANTIDAD DE REGISTROS TOTALES POR FUENTE
dfs = {'museos':museos, 'cines':cines, 'bibliotecas':bibliotecas}
lst = list()
for name, df in dfs.items():
    lst.append({'source': name, 'count':df.size})
size_by_source = pd.DataFrame(lst)
size_by_source

,source,count
0,museos,28368
1,cines,8554
2,bibliotecas,50425


In [98]:
#CANTIDAD DE REGISTROS POR PROVINCIA Y CATEGORIA
size_by_prov_cat = df.groupby(["provincia", "categoría"], as_index=False).size()
size_by_prov_cat

,provincia,categoría,size
0,Buenos Aires,Bibliotecas Populares,543
1,Buenos Aires,Espacios de Exhibición Patrimonial,152
2,Buenos Aires,Salas de cine,106
3,Catamarca,Bibliotecas Populares,38
4,Catamarca,Espacios de Exhibición Patrimonial,32
...,...,...,...
68,Tierra del Fuego,Salas de cine,3
69,"Tierra del Fuego, Antártida e Islas del Atlánt...",Espacios de Exhibición Patrimonial,14
70,Tucumán,Bibliotecas Populares,37
71,Tucumán,Espacios de Exhibición Patrimonial,26


In [100]:
#Procesar la informacion de cines para poder crear una tabla que contenga Provincia, Cant Pantallas, Butacas y Espacio Incaa
cines['espacio_INCAA'] = cines['espacio_INCAA'].fillna(0)
cines['espacio_INCAA'] = cines['espacio_INCAA'].replace(['si', 'SI'], '1')
cines['espacio_INCAA'] = cines['espacio_INCAA'].astype('int64')
cines_insights = cines.groupby("provincia")["Pantallas", "Butacas", "espacio_INCAA"].sum()
cines_insights.head()

C:\Users\Juan Cruz\AppData\Local\Temp\ipykernel_15268\3007675943.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  cines_insights = cines.groupby("provincia")["Pantallas", "Butacas", "espacio_INCAA"].sum()


,Pantallas,Butacas,espacio_INCAA
provincia,,,
Buenos Aires,358,93112,20
Catamarca,12,3200,1
Chaco,14,2469,1
Chubut,10,2682,4
Ciudad Autónoma de Buenos Aires,153,31386,3
